# Utils

In [30]:
from itertools import product
from typing import Tuple, Literal, Callable

import numpy as np
import numpy.linalg as linalg
from pandarallel import pandarallel
import pandas as pd
import plotly.graph_objects as go

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [31]:
def _swap_rows(matrix: np.ndarray, i: int, j: int) -> None:
    matrix[[i, j]] = matrix[[j, i]]


def get_LUP_decomposition(matrix: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    if linalg.det(matrix) == 0:
        raise ValueError('Singular matrix')

    size = matrix.shape[0]

    LU = matrix.copy()
    P = np.eye(size)

    for column_index in range(size):
        pivot_index = abs(LU[column_index:, column_index]).argmax() + column_index

        _swap_rows(P, column_index, pivot_index)
        _swap_rows(LU, column_index, pivot_index)

        for row_index in range(column_index + 1, size):
            LU[row_index, column_index] /= LU[column_index, column_index]
            for k in range(column_index + 1, size):
                LU[row_index][k] -= LU[row_index][column_index] * LU[column_index][k]

    L = np.eye(size)
    U = np.zeros((size, size))

    for row_index in range(size):
        for column_index in range(size):
            if row_index <= column_index:
                U[row_index][column_index] = LU[row_index][column_index]
            else:
                L[row_index][column_index] = LU[row_index][column_index]

    return L, U, P

In [32]:
def _forward_substitution(L: np.ndarray, right_part: np.ndarray) -> np.ndarray:
    size = L.shape[0]

    solution = []
    for i in range(size):
        solution.append((right_part[i] - np.dot(L[i, :i], solution).sum()) / L[i][i])

    return np.array(solution)


def _back_substitution(U: np.ndarray, right_part: np.ndarray) -> np.ndarray:
    return _forward_substitution(np.flip(U, axis=[0, 1]), right_part[::-1])[::-1]


def solve_using_LU_decompostion(matrix: np.ndarray, right_part: np.ndarray) -> np.ndarray:
    L, U, P = get_LUP_decomposition(matrix)
    y = _forward_substitution(L, P.dot(right_part))
    return _back_substitution(U, y)

In [33]:
def calculate_spectral_condition_number(matrix: np.ndarray) -> float:
    return linalg.cond(matrix)


def calculate_volume_condition_number(matrix: np.ndarray) -> float:
    volume_0 = abs(linalg.det(matrix))  # Объём косоугольного параллелепипеда

    volume = 1  # Объём прямоугольного параллелепипеда
    for row in matrix:
        volume *= linalg.norm(row)

    return volume / volume_0


def calculate_angle_condition_number(matrix: np.ndarray) -> float:
    inverse_matrix = linalg.inv(matrix)

    candidates = []
    for row, column in zip(matrix, inverse_matrix.T):
        candidates.append(linalg.norm(row) * linalg.norm(column))

    return max(candidates)

In [34]:
def get_condition_number_function_by_name(name: Literal['spectral', 'volume', 'angle']) -> Callable[[np.ndarray], float]:
    if name == 'spectral':
        condition_function = calculate_spectral_condition_number
    elif name == 'volume':
        condition_function = calculate_volume_condition_number
    else:
        condition_function = calculate_angle_condition_number

    return condition_function

In [35]:
def banchmark_linear_system(
        matrix: np.ndarray,
        right_part: np.ndarray,
        true_solution: np.ndarray,
        criterion: Literal['spectral', 'volume', 'angle'] = 'angle',
) -> pd.Series:
    if linalg.det(matrix) == 0:
        return None

    condition_function = get_condition_number_function_by_name(criterion)

    L, U, P = get_LUP_decomposition(matrix)

    matrix_condition_number = condition_function(matrix)
    L_condition_number = condition_function(L)
    U_condition_number = condition_function(U)

    actual_solution = solve_using_LU_decompostion(matrix, right_part)

    return pd.Series(
        [
            matrix_condition_number,
            L_condition_number,
            U_condition_number,
            linalg.norm(true_solution - actual_solution),
        ]
    )

In [36]:
def generate_matrix(element_factory: Callable[[int, int], float], size: int) -> np.ndarray:
    return np.array(
        [
            [element_factory(row, column) for column in range(1, size + 1)]
            for row in range(1, size + 1)
        ],
        dtype=float,
    )

In [37]:
def compare_n_and_cond(data: pd.DataFrame) -> None:
    fig = go.Figure()

    fig.add_scatter(x=data['n'], y=data['cond(A)'], name='cond(A)')
    fig.add_scatter(x=data['n'], y=data['cond(L)'], name='cond(L)')
    fig.add_scatter(x=data['n'], y=data['cond(U)'], name='cond(U)')

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Число обусловленности', tickformat='.2e')
    fig.update_layout(title='Зависимость числа обусловленности от размера матрицы')

    fig.show()

In [38]:
def compare_n_and_error(data: pd.DataFrame) -> None:
    fig = go.Figure()

    fig.add_scatter(x=data['n'], y=data['error'])

    fig.update_xaxes(title='Размер матрицы')
    fig.update_yaxes(title='Ошибка', tickformat='.2e')
    fig.update_layout(title='Зависимость ошибки от размера матрицы')

    fig.show()

# Diagonally dominant

In [39]:
def get_diagonally_dominant_linear_system(size: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    matrix = np.eye(size) * 2 + np.eye(size, k=-1) * -1 + np.eye(size, k=1) * -1
    solution = np.ones(size)
    right_part = np.dot(matrix, solution)
    return matrix, right_part, solution

In [40]:
diagonally_dominant_data = pd.DataFrame(range(2, 201), columns=['n'])
diagonally_dominant_data[['cond(A)', 'cond(L)', 'cond(U)', 'error']] = diagonally_dominant_data.parallel_apply(
    lambda row: banchmark_linear_system(*get_diagonally_dominant_linear_system(row.n)),
    axis=1,
)
diagonally_dominant_data

,n,cond(A),cond(L),cond(U),error
0,2,1.666667,1.118034,1.118034,0.000000e+00
1,3,3.000000,1.343710,1.343710,1.110223e-16
2,4,3.949684,1.502313,1.559024,1.110223e-16
3,5,5.338539,1.666417,1.753568,1.110223e-16
4,6,6.546537,1.804701,1.930745,1.110223e-16
...,...,...,...,...,...
194,196,977.584587,9.912135,11.416437,1.263627e-13
195,197,985.062688,9.937325,11.445597,1.267409e-13
196,198,992.509383,9.962450,11.474683,1.277135e-13
197,199,1000.025000,9.987513,11.503696,1.275921e-13


In [41]:
compare_n_and_cond(diagonally_dominant_data)

Unsupported

In [42]:
compare_n_and_error(diagonally_dominant_data)

Unsupported

# Hilbert

In [43]:
def get_hilbert_linear_system(size: int) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    matrix = generate_matrix(lambda row, column: 1 / (row + column - 1), size)
    solution = np.ones(size)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [44]:
hilbert_data = pd.DataFrame(range(2, 29), columns=['n'])
hilbert_data[['cond(A)', 'cond(L)', 'cond(U)', 'error']] = hilbert_data.apply(
    lambda row: banchmark_linear_system(*get_hilbert_linear_system(row.n)),
    axis=1,
)
hilbert_data

,n,cond(A),cond(L),cond(U),error
0,2,8.062258e+00,1.118034,1.118034e+00,8.005932e-16
1,3,1.728872e+02,1.500000,1.634693e+00,1.093443e-15
2,4,4.020913e+03,2.173067,2.660409e+00,3.023827e-13
3,5,9.515770e+04,2.438588,4.304053e+00,8.289747e-13
4,6,2.441571e+06,2.388348,7.774785e+00,6.216114e-10
5,7,7.162186e+07,2.669159,1.396505e+01,3.432606e-08
6,8,2.025215e+09,2.815274,2.521972e+01,3.393981e-07
7,9,5.577437e+10,2.903601,4.747207e+01,4.408102e-05
8,10,1.719868e+12,3.213568,8.902259e+01,8.471537e-04
9,11,5.240136e+13,3.314750,1.660323e+02,1.523856e-02


In [45]:
compare_n_and_cond(hilbert_data)

Unsupported

In [46]:
compare_n_and_error(hilbert_data)

Unsupported

# Pakulina

In [47]:
def get_pakulina_1_linear_system() -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    matrix = np.array([
        [3.278164, 1.046583, -1.378574],
        [1.046583, 2.975937, 0.934251],
        [-1.378574, 0.934251, 4.836173],
    ])
    solution = np.ones(3)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [48]:
def get_pakulina_7_linear_system() -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    matrix = np.array([
        [9.331343, 1.120045, -2.880925],
        [1.120045, 7.086042, 0.670297],
        [-2.880925, 0.670297, 5.622534],
    ])
    solution = np.ones(3)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [49]:
def get_pakulina_14_linear_system() -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    matrix = np.array([
        [9.016024, 1.082197, -2.783575],
        [1.082197, 4.543595, 0.647647],
        [-2.783575, 0.647647, 5.432541],
    ])
    solution = np.ones(3)
    right_part = matrix.dot(solution)
    return matrix, right_part, solution

In [50]:
pakulina_data = pd.DataFrame.from_dict({
    1: banchmark_linear_system(*get_pakulina_1_linear_system()),
    7: banchmark_linear_system(*get_pakulina_7_linear_system()),
    14: banchmark_linear_system(*get_pakulina_14_linear_system()),
}, orient='index')
pakulina_data.columns = ['cond(A)', 'cond(L)', 'cond(U)', 'error']
pakulina_data

,cond(A),cond(L),cond(U),error
1,1.906892,1.203120,1.270739,7.447602e-16
7,1.502929,1.058710,1.061991,0.000000e+00
14,1.575103,1.069951,1.077958,0.000000e+00


# Regularization

In [51]:
def get_regularization_data(size: int, alpha: float, criterion: Literal['spectral', 'volume', 'angle'] = 'angle') -> pd.Series:
    matrix, right_part, true_solution = get_hilbert_linear_system(size)

    actual_solution_without_regularization = solve_using_LU_decompostion(matrix, right_part)

    regularized_matrix = matrix + alpha * np.eye(matrix.shape[0])
    actual_solution_with_regularization = solve_using_LU_decompostion(regularized_matrix, right_part)

    condition_number_function = get_condition_number_function_by_name(criterion)

    matrix_condition_number = condition_number_function(matrix)
    regularized_matrix_condition_matrix = condition_number_function(regularized_matrix)

    return pd.Series([
        matrix_condition_number, 
        regularized_matrix_condition_matrix,
        linalg.norm(true_solution - actual_solution_without_regularization),
        linalg.norm(true_solution - actual_solution_with_regularization),
    ])

In [52]:
regularization_data = pd.DataFrame(product(range(2, 29), [10 ** -i for i in range(1, 13)]), columns=['n', 'alpha'])
regularization_data[['cond(A)', 'cond(A - alpha * E)', 'error', 'error_r']] = regularization_data.parallel_apply(
    lambda row: get_regularization_data(int(row.n), row.alpha),
    axis=1,
)
regularization_data

,n,alpha,cond(A),cond(A - alpha * E),error,error_r
0,2,1.000000e-01,8.062258e+00,3.527083e+00,8.005932e-16,0.266335
1,2,1.000000e-02,8.062258e+00,7.063907e+00,8.005932e-16,0.055135
2,2,1.000000e-03,8.062258e+00,7.948886e+00,8.005932e-16,0.006232
3,2,1.000000e-04,8.062258e+00,8.050765e+00,8.005932e-16,0.000632
4,2,1.000000e-05,8.062258e+00,8.061107e+00,8.005932e-16,0.000063
...,...,...,...,...,...,...
319,28,1.000000e-08,7.543538e+16,2.973126e+07,2.060809e+03,0.000299
320,28,1.000000e-09,7.543538e+16,2.846667e+08,2.060809e+03,0.000093
321,28,1.000000e-10,7.543538e+16,2.704007e+09,2.060809e+03,0.000030
322,28,1.000000e-11,7.543538e+16,2.533369e+10,2.060809e+03,0.000069


In [53]:
def find_best_alpha(size: int, data: pd.DataFrame) -> pd.Series:
    data = data[data.n == size]
    data.reset_index(drop=True, inplace=True)

    min_error_arg = data['error_r'].argmin()

    min_error = data.loc[min_error_arg, 'error_r']
    alpha = data.loc[min_error_arg, 'alpha']

    return pd.Series([data.loc[min_error_arg, 'cond(A)'], data.loc[min_error_arg, 'cond(A - alpha * E)'], alpha, min_error])

def find_min_error(size: int, data: pd.DataFrame) -> float:
    return data[data.n == size]['error'].min()

In [54]:
regularization_best_data = pd.DataFrame(range(2, 29), columns=['n'])
regularization_best_data[['cond(A)', 'cond(A - alpha * E)', 'alpha_r', 'min_error_r']] = regularization_best_data.apply(
    func=lambda row: find_best_alpha(row.n, regularization_data),
    axis=1,
)
regularization_best_data['min_error'] = regularization_best_data.apply(
    func=lambda row: find_min_error(row.n, regularization_data),
    axis=1,
)
regularization_best_data

,n,cond(A),cond(A - alpha * E),alpha_r,min_error_r,min_error
0,2,8.062258e+00,8.062258e+00,1.000000e-12,6.324345e-12,8.005932e-16
1,3,1.728872e+02,1.728872e+02,1.000000e-12,3.854773e-11,1.093443e-15
2,4,4.020913e+03,4.020913e+03,1.000000e-12,2.356882e-10,3.023827e-13
3,5,9.515770e+04,9.515767e+04,1.000000e-12,1.471606e-09,8.289747e-13
4,6,2.441571e+06,2.441549e+06,1.000000e-12,8.043307e-09,6.216114e-10
5,7,7.162186e+07,7.160136e+07,1.000000e-12,6.152154e-08,3.432606e-08
6,8,2.025215e+09,2.007158e+09,1.000000e-12,1.032176e-06,3.393981e-07
7,9,5.577437e+10,4.337934e+10,1.000000e-12,5.448681e-06,4.408102e-05
8,10,1.719868e+12,1.695058e+11,1.000000e-12,9.742020e-07,8.471537e-04
9,11,5.240136e+13,2.056514e+10,1.000000e-11,1.053211e-05,1.523856e-02


In [55]:
fig = go.Figure()

fig.add_scatter(
    x=regularization_best_data['n'],
    y=regularization_best_data['cond(A)'],
    name='cond(A)',
)
fig.add_scatter(
    x=regularization_best_data['n'],
    y=regularization_best_data['cond(A - alpha * E)'],
    name='cond(A - alpha * E)',
)

fig.update_xaxes(title='Размер матрицы')
fig.update_yaxes(title='Число обусловленности', tickformat='.2e')
fig.update_layout(title='Зависимость числа обусловленности от размера матрицы')

fig.show()

Unsupported

In [56]:
fig = go.Figure()

fig.add_scatter(x=regularization_best_data.n, y=regularization_best_data.min_error, name='Без регуляризации')
fig.add_scatter(x=regularization_best_data.n, y=regularization_best_data.min_error_r, name='С регуляразиацией')

fig.update_xaxes(title='Размер матрицы')
fig.update_yaxes(title='Ошибка', tickformat='.2e')
fig.update_layout(title='Зависимость наименьшей ошибки от размера матрицы')

fig.show()

Unsupported

In [57]:
def test_alpha(size: int, alpha: float, with_regularization: bool = True):
    matrix = generate_matrix(lambda row, column: 1 / (row + column - 1), size)
    true_solution = np.random.rand(size)
    right_part = matrix.dot(true_solution)

    if with_regularization:
        matrix = matrix + alpha * np.eye(matrix.shape[0])

    actual_solution = solve_using_LU_decompostion(matrix, right_part)

    print(f'n = {size}, {alpha = }: {linalg.norm(true_solution - actual_solution)} ({with_regularization })')

In [58]:
test_alpha(5, 1e-12)
test_alpha(5, 1e-12, with_regularization=False)

test_alpha(11, 1e-11)
test_alpha(11, 1e-11, with_regularization=False)

test_alpha(28, 1e-10)
test_alpha(28, 1e-10, with_regularization=False)

n = 5, alpha = 1e-12: 6.834821277860977e-08 (True)
n = 5, alpha = 1e-12: 9.8638028432819e-12 (False)
n = 11, alpha = 1e-11: 0.3107472765633403 (True)
n = 11, alpha = 1e-11: 0.011142244145831552 (False)
n = 28, alpha = 1e-10: 1.1051157529828148 (True)
n = 28, alpha = 1e-10: 906.1711169706781 (False)
